In [1]:
import cv2
import os
import pandas as pd

# Resizing images

We resize images not to be too large for the model to handle. It is also useful as we use the size 640x480 for the model, so we do not need to resize the dataset each time we load it.

In [2]:
df = pd.read_csv("test_annotation.txt", header=None, names=["filename", "xmin", "ymin", "xmax", "ymax", "class"])

In [3]:
df = df[df["class"].isin(["Rocket", "Potato"])]

In [4]:
from pathlib import Path
from tqdm import tqdm

H, W = 480, 640

resized = Path("resized")
resized.mkdir(exist_ok=True, parents=True)
(resized / "train").mkdir(exist_ok=True, parents=True)
(resized / "test").mkdir(exist_ok=True, parents=True)

for idx, row in tqdm(df.iterrows()):
    if not Path(row['filename']).exists():
        df.drop(idx, inplace=True)
        continue
    img = cv2.imread(row['filename'])
    img_h, img_w = img.shape[:2]
    img = cv2.resize(img, (W, H))
    df.loc[idx, 'xmin'] = int(row['xmin'] * W / img_w)
    df.loc[idx, 'ymin'] = int(row['ymin'] * H / img_h)
    df.loc[idx, 'xmax'] = int(row['xmax'] * W / img_w)
    df.loc[idx, 'ymax'] = int(row['ymax'] * H / img_h)
    cv2.imwrite(os.path.join("resized", row['filename']), img)

285it [00:03, 90.60it/s] 


In [5]:
df.to_csv("resized/test_annotation.txt", header=False, index=False)